In [26]:
import pandas as pd
import gdown
import sqlite3 as sql3

In [27]:
url='https://drive.google.com/file/d/1LCpP0FD-JFwJ0V0k2wbi15Q7s3goGZfa/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
gdown.download(url, 'chinook.db', quiet=False)
connection = sql3.connect('chinook.db')

Downloading...
From: https://drive.google.com/uc?id=1LCpP0FD-JFwJ0V0k2wbi15Q7s3goGZfa
To: c:\Users\mail\OneDrive\Documents\my_python\zooming_up\SQL\chinook.db
100%|██████████| 885k/885k [00:00<00:00, 3.76MB/s]


In [28]:
sql_query = """
            SELECT *
            FROM sqlite_master
            WHERE type = 'table'
            """
cursor = connection.cursor()
cursor.execute(sql_query)
a = cursor.fetchall()
t = []
for i in range(12):
    try:
        t.append(a[i][4].split("\"")[1])
    except IndexError:
        continue
t

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks']

JOIN через WHERE

In [65]:
query = """
        SELECT      *

        -- в таблице albums 347 записей, в таблице artists - 275

        FROM        artists, albums

        -- перечислением через запятую мы делаем cross join или декартово произведение  таблиц
        -- порядок не имеет значения - для каждой записи из одной таблицы добавляются все записи из второй и общее число записей получается 95425

        WHERE albums.artistid = artists.artistid

        -- из cross join мы фильтруем только те записи, в которых совпадают ключи и таким образом превращаем cross join в inner join
        """
pd.read_sql_query(query, connection)

,ArtistId,Name,AlbumId,Title,ArtistId
0,1,AC/DC,1,For Those About To Rock We Salute You,1
1,2,Accept,2,Balls to the Wall,2
2,2,Accept,3,Restless and Wild,2
3,1,AC/DC,4,Let There Be Rock,1
4,3,Aerosmith,5,Big Ones,3
...,...,...,...,...,...
342,226,Eugene Ormandy,343,Respighi:Pines of Rome,226
343,272,Emerson String Quartet,344,Schubert: The Late String Quartets & String Qu...,272
344,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",345,Monteverdi: L'Orfeo,273
345,274,Nash Ensemble,346,Mozart: Chamber Music,274


Выбираем только неуникальные значения

In [11]:
query = """
        SELECT      name, count(1) as cnt
        FROM        tracks
        GROUP BY    name
        HAVING      count(1)>1
        ORDER BY    cnt DESC
        LIMIT 3
        """
pd.read_sql_query(query, connection)

,Name,cnt
0,Wrathchild,5
1,The Trooper,5
2,The Number Of The Beast,5


При группировке мы можем вывести только постоянное значение, поле группировки и агрегаты групп, получившихся в результатах группировки

In [17]:
query = """
        SELECT      'test' as literal, composer, sum(UnitPrice) as sum
        FROM        tracks
        GROUP BY    composer 
        ORDER BY    sum DESC
        LIMIT 5
        """
pd.read_sql_query(query, connection)

,literal,Composer,sum
0,test,None,1181.22
1,test,Steve Harris,79.20
2,test,U2,43.56
3,test,Jagger/Richards,34.65
4,test,Billy Corgan,30.69


Порядок выполнения, последним заполняем SELECT

In [ ]:
query = """
        SELECT      
        FROM
            JOIN
        WHERE        
        GROUP BY
        HAVING    
        ORDER BY    
        LIMIT
        """